In [ ]:
!pip install --quiet datasets
import pandas as pd
from datasets import list_datasets,load_dataset


     |████████████████████████████████| 325 kB 39.6 MB/s 
     |████████████████████████████████| 77 kB 8.0 MB/s 
     |████████████████████████████████| 136 kB 75.0 MB/s 
     |████████████████████████████████| 1.1 MB 63.6 MB/s 
     |████████████████████████████████| 212 kB 75.8 MB/s 
     |████████████████████████████████| 127 kB 27.8 MB/s 
     |████████████████████████████████| 271 kB 71.9 MB/s 
     |████████████████████████████████| 94 kB 2.3 MB/s 
     |████████████████████████████████| 144 kB 70.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


Load multidoc2dial dataset

In [ ]:
dataset=load_dataset("multidoc2dial",name="multidoc2dial")

Generating validation split:   0%|          | 0/4201 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/21451 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5 [00:00<?, ? examples/s]

Dataset multi_doc2dial downloaded and prepared to /root/.cache/huggingface/datasets/multi_doc2dial/multidoc2dial/1.0.0/6e2a407c09eb478a5b80845fc04406bb9c9d4bea9f135dd7b8b7610a6b608d84. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Change to dataframe and use dmv subset and perform cleaning


> Indented block



In [ ]:
df=pd.DataFrame(dataset['train'])
dmv=df[df['domain']=='dmv']

In [ ]:
import re
dmv['contextClean']=dmv['context'].apply(lambda x:re.sub(r'\[.*?\]','',x))
dmv['contextClean']=dmv['contextClean'].apply(lambda x:re.sub(r'[^\w\s]','',x))
# dmv['contextClean']=dmv['context'].apply(lambda x:re.sub(r'[\(\[].*?[\)\]]','',x))
dmv['contextClean']=dmv['contextClean'].apply(lambda x:re.sub(r'[\n,*]','',x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [ ]:
dmv['contextClean']=dmv['contextClean'].apply(lambda x:re.sub(r'[\n,*-:]','',x))
dmv['contextClean']=dmv['contextClean'].apply(lambda x:re.sub(r'\s+',' ',x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Get unique context and its length

In [ ]:
len(dmv['contextClean'].unique())

127

In [ ]:
context=pd.Series(dmv['contextClean'].unique())

In [ ]:
len(dmv)

6135

In [ ]:
dmv['questionSplit']=dmv['question'].str.split("[").apply(lambda x: x[0])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Import faiss search and sentence transformers

In [ ]:
!pip install --quiet faiss-cpu faiss-gpu
! pip install --quiet sentence-transformers
import numpy as np
import faiss

     |████████████████████████████████| 8.6 MB 1.0 MB/s 
     |████████████████████████████████| 85.5 MB 93 kB/s 
     |████████████████████████████████| 79 kB 3.7 MB/s 
     |████████████████████████████████| 4.0 MB 12.0 MB/s 
     |████████████████████████████████| 1.2 MB 38.3 MB/s 
     |████████████████████████████████| 6.6 MB 31.6 MB/s 
     |████████████████████████████████| 895 kB 36.4 MB/s 
     |████████████████████████████████| 596 kB 43.6 MB/s 


Load Assymetric search model

In [ ]:
from sentence_transformers import SentenceTransformer
model=SentenceTransformer('msmarco-distilbert-base-dot-prod-v3')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/115 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Use faiss index to index context and store them

In [ ]:
encode_contexts=model.encode(context.to_list())
encode_contexts=np.asarray(encode_contexts.astype('float32'))
index=faiss.IndexIDMap(faiss.IndexFlatIP(768))
index.add_with_ids(encode_contexts,np.array(range(0,len(context))))

Write index to file to use it later

In [ ]:
faiss.write_index(index,"context.index")  # save the index to disk
index = faiss.read_index("context.index")  # load the index


Search context for given query

In [ ]:
def search(query,top_k,index,model):
  query_vector=model.encode([query])
  top_k=index.search(query_vector,top_k)
  top_k_ids=top_k[1].tolist()[0]
  top_k_ids=list(np.unique(top_k_ids))
  results=[context.iloc[id] for id in top_k_ids]
  return results,top_k_ids

In [ ]:
res=search(query,3,index,model)

Create Dataframe to store result

In [ ]:
output=pd.DataFrame(columns=['query','output','id','exp_o','acc'])

Add context for each query

In [ ]:
for i in range(len(dmv['question'])):
  res=search(dmv['questionSplit'][i],3,index,model)
  output=output.append({'query':dmv['question'][i],'output':res[0][0],'id':res[1],'exp_o':dmv['contextClean'][i],'acc':dmv['contextClean'][i] in res[0]},ignore_index=True)

In [ ]:
output.to_csv('resultsSplit.csv')

In [ ]:
output.head()

,query,output,id,exp_o,acc
0,"Hello, I forgot o update my address, can you h...",How to renew a registration You need to renew ...,"[30, 67, 124]",Many DMV customers make easily avoidable mista...,False
1,Can I do my DMV transactions online?[SEP]agent...,Renew a business license or registration IMPOR...,"[73, 99, 116]",Many DMV customers make easily avoidable mista...,False
2,You've got it. Another query about DMV. What h...,Many DMV customers make easily avoidable mista...,"[0, 8, 122]",What is a suspension for failure to pay tolls ...,False
3,"Besides that, will I receive a notice?[SEP]age...",Pay insurance lapse civil penalty Determine if...,"[11, 59, 61]",What is a suspension for failure to pay tolls ...,False
4,If you submit the affidavit?[SEP]agent: the N...,What do I need to do with my plates if I move ...,"[9, 21, 69]",What is a suspension for failure to pay tolls ...,False


In [ ]:
output['acc'].value_counts() #with Split

False    3347
True     2788
Name: acc, dtype: int64

In [ ]:
70%accuracy#with question